To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Unsloth supports Qwen2 0.5b, 1.5b. 7b, and the largest 72b!**

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

* Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes 등 다양한 모델 지원
* 16bit LoRA or 4bit QLoRA 등 트레이닝 방법 지원

In [13]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/Qwen2-0.5b-bnb-4bit",           # Qwen2 2x faster!
    "unsloth/Qwen2-1.5b-bnb-4bit",
    "unsloth/Qwen2-7b-bnb-4bit",
    "unsloth/Qwen2-72b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # Try more models at https://huggingface.co/unsloth!

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b-bnb-4bit", # Reminder we support ANY Hugging Face model!
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.9.post1: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [14]:
model

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): GemmaFixedRotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear4bit(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear4bit(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSN

* LoRA를 이용하여 모델 튜닝

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, #16, 32
    lora_dropout = 0.03, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3401,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.03.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.9.post1 patched 26 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [4]:
alpaca_prompt = """<bos><start_of_turn>user
{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn>"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


In [5]:

from datasets import load_dataset
dataset = load_dataset("mkh0813/dataset_for_education", split = "train")
print(dataset[0])
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/345 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/42.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/366 [00:00<?, ? examples/s]

{'instruction': '당신은 사용자의 질문에 친절하게 대답을 합니다.', 'input': '그리드원에 대해서 알려주세요.', 'output': "그리드원은 2005년에 설립된 소프트웨어 전문 기업으로, 로보틱 처리 자동화(RPA)와 생성형 인공지능(AI) 기반 업무 자동화 솔루션을 제공합니다. 2017년 국내 최초 RPA 솔루션 'AutomateOne'을 개발하였고, 2023년에는 생성형 AI 기반의 업무 자동화 솔루션 'GO Automation Platform'을 출시했습니다. 이 회사는 다양한 금융, 공공, 기업을 대상으로 업무 효율성을 높이는 자동화 솔루션을 개발하고 있습니다."}


Map:   0%|          | 0/366 [00:00<?, ? examples/s]

In [6]:
dataset[2]

{'instruction': '사용자의 질문에는 언제나 친절한 답변으로 응대하세요.',
 'input': '그리드원이란 무엇인가요?',
 'output': "2005년 설립된 소프트웨어 전문 기업 그리드원은 RPA 및 생성형 AI 기반 업무 자동화 솔루션으로 다양한 산업 분야를 지원합니다. 국내 최초 RPA 솔루션 'AutomateOne'을 2017년에 선보인 그리드원은 2023년에는 생성형 AI 기반 업무 자동화 솔루션 'GO Automation Platform'을 출시했습니다.  금융, 공공, 기업 등 다양한 고객들에게 업무 효율성 향상을 위한 자동화 솔루션을 제공합니다.",
 'text': "<bos><start_of_turn>user\n그리드원이란 무엇인가요?<end_of_turn>\n<start_of_turn>model\n2005년 설립된 소프트웨어 전문 기업 그리드원은 RPA 및 생성형 AI 기반 업무 자동화 솔루션으로 다양한 산업 분야를 지원합니다. 국내 최초 RPA 솔루션 'AutomateOne'을 2017년에 선보인 그리드원은 2023년에는 생성형 AI 기반 업무 자동화 솔루션 'GO Automation Platform'을 출시했습니다.  금융, 공공, 기업 등 다양한 고객들에게 업무 효율성 향상을 위한 자동화 솔루션을 제공합니다.<end_of_turn><eos>"}

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 3000,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        # optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/366 [00:00<?, ? examples/s]

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 366 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 135
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss
1,3.775000
2,2.964800
3,3.266800
4,3.054400
5,2.878400
6,2.425100
7,2.075900
8,1.946300
9,2.025200
10,2.034300


<a name="Inference"></a>
### 추론하기
Let's run the model! You can change the instruction and input - leave the output blank!

In [12]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.03, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [9]:
alpaca_prompt = """<start_of_turn>user
{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn>"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass



# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!
inputs = tokenizer(
[
    alpaca_prompt.format(
        "그리드원에 대해 알려주세요.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

<bos><start_of_turn>user
그리드원에 대해 알려주세요.<end_of_turn>
<start_of_turn>model
<end_of_turn>
2005년 설립된 소프트웨어 전문 기업 그리드원은 RPA 및 생성형 AI 기반 업무 자동화 솔루션을 제공합니다. 국내 최초 RPA 솔루션 'AutomateOne'을 2017년에 개발했으며, 2023년에는 생성형 AI 기반 업무 자동화 솔루션 'GO Automation Platform'을 출시했습니다.  그리드원은 이러한 솔루션을 통해 다양한 업무 자동화를 가능하게 합니다.<end_of_turn><eos>


In [10]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!
inputs = tokenizer(
[
    alpaca_prompt.format(
        "폴리텍대학교에 대해 알려주세요.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

<bos><start_of_turn>user
폴리텍대학교에 대해 알려주세요.<end_of_turn>
<start_of_turn>model
<end_of_turn>
전국 곳곳에 8개 대학, 35개 캠퍼스, 1개 대안학교를 갖춘 폴리텍대학교는 1977년 설립 이후 실무 중심의 기술 교육을 통해 다양한 산업 분야의 전문 인력을 양성해왔습니다.<end_of_turn><eos>


In [11]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Unsloth has 2x faster inference!
inputs = tokenizer(
[
    alpaca_prompt.format(
        "자기소개 부탁드려요", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

<bos><start_of_turn>user
자기소개 부탁드려요<end_of_turn>
<start_of_turn>model
<end_of_turn>
한국폴리텍대학교는 1977년 설립된 기술 대학으로, 실무 중심의 기술 교육을 통해 다양한 산업 분야의 전문 인력을 양성하고 있습니다. 이를 통해 높은 취업률을 달성하고 있습니다.<end_of_turn><eos>
